In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
import torch

import sys
sys.path.insert(1,"../../midterm_election_semantics_2022/")
from helpers import clean_tweets

import os
os.environ['TRANSFORMERS_CACHE'] = '../../../election_candidate_sematics/cache'

In [2]:
from transformers import AutoModelForSequenceClassification, pipeline
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax

MODEL = f"cardiffnlp/xlm-twitter-politics-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

Downloading: 100%|██████████| 643/643 [00:00<00:00, 322kB/s]
c:\Users\thatb\anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\thatb\OneDrive\Desktop\Fall 2022\Data Driven Policy\election_candidate_sematics\cache. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 5

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Downloading: 100%|██████████| 975/975 [00:00<00:00, 484kB/s]
Downloading: 100%|██████████| 1.11G/1.11G [00:46<00:00, 23.8MB/s]


In [4]:
climate_tweets = pd.read_csv("./data/climate_tweets_11_1.csv")
loan_tweets = pd.read_csv("./data/loan_tweets_11_1.csv")

In [21]:
climate_tweets = climate_tweets.loc[~climate_tweets['clean_text'].isna()]
loan_tweets = loan_tweets.loc[~loan_tweets['clean_text'].isna()]

In [5]:
climate_tweets.head()

,Unnamed: 0,id,tweet_url,created_at,parsed_created_at,user_screen_name,text,tweet_type,coordinates,hashtags,...,user_friends_count,user_listed_count,user_location,user_name,user_statuses_count,user_time_zone,user_urls,user_verified,clean_text,topic_id
0,74,1587226545445273602,https://twitter.com/ChrisVanHollen/status/1587...,Mon Oct 31 23:34:32 +0000 2022,2022-10-31T23:34:32+00:00,ChrisVanHollen,Great to celebrate the launch of Montgomery Co...,original,NaN,NaN,...,963,2566,"Maryland, USA",Senator Chris Van Hollen,7425,NaN,https://www.vanhollen.senate.gov/,True,Great to celebrate the launch of Montgomery Co...,615
1,676,1586459473920475136,https://twitter.com/PattyMurray/status/1586459...,Sat Oct 29 20:46:28 +0000 2022,2022-10-29T20:46:28+00:00,PattyMurray,The climate crisis is one of the most serious ...,original,NaN,NaN,...,2392,3921,WA & DC,Senator Patty Murray,23406,NaN,http://murray.senate.gov/,True,The climate crisis is one of the most serious ...,26
2,772,1586416039038160897,https://twitter.com/RepValDemings/status/15864...,Sat Oct 29 17:53:52 +0000 2022,2022-10-29T17:53:52+00:00,RepValDemings,Florida families cannot continue to have their...,original,NaN,NaN,...,895,2205,"Washington, DC I Orlando, FL",Rep. Val Demings,14028,NaN,http://demings.house.gov,True,Florida families cannot continue to have their...,26
3,1421,1517212239270404097,https://twitter.com/salviforsenate/status/1517...,Thu Apr 21 18:42:40 +0000 2022,2022-04-21T18:42:40+00:00,salviforsenate,https://t.co/LMDX527Iid,reply,NaN,NaN,...,221,21,NaN,Kathy Salvi,577,NaN,http://kathysalvi.com,True,NaN,758
4,1560,1572733928867258375,https://twitter.com/TammyDuckworth/status/1572...,Wed Sep 21 23:46:03 +0000 2022,2022-09-21T23:46:03+00:00,TammyDuckworth,🚨🌎This is a BIG deal for our planet. The Kiga...,quote,NaN,NaN,...,3234,1209,Illinois,Tammy Duckworth,4651,NaN,http://tammyduckworth.com/,True,policecarlight globeshowingAmericas This is ...,26


In [6]:
loan_tweets.head()

,Unnamed: 0,id,tweet_url,created_at,parsed_created_at,user_screen_name,text,tweet_type,coordinates,hashtags,...,user_friends_count,user_listed_count,user_location,user_name,user_statuses_count,user_time_zone,user_urls,user_verified,clean_text,topic_id
0,23,1587124150832959490,https://twitter.com/PattyMurray/status/1587124...,Mon Oct 31 16:47:39 +0000 2022,2022-10-31T16:47:39+00:00,PattyMurray,TODAY is the last day to apply for Public Serv...,quote,NaN,NaN,...,2392,3914,WA & DC,Senator Patty Murray,23410,NaN,http://murray.senate.gov/,True,TODAY is the last day to apply for Public Serv...,575
1,427,1587157154846449670,https://twitter.com/JamesForAR/status/15871571...,Mon Oct 31 18:58:48 +0000 2022,2022-10-31T18:58:48+00:00,JamesForAR,Today is the LAST DAY to apply for the Public ...,retweet,NaN,NaN,...,14738,121,"Arkansas, USA",Natalie James for U.S. Senate,3424,NaN,http://JamesforArkansas.com,True,Today is the LAST DAY to apply for the Public ...,575
2,471,1586761427930550272,https://twitter.com/SenDuckworth/status/158676...,Sun Oct 30 16:46:19 +0000 2022,2022-10-30T16:46:19+00:00,SenDuckworth,🚨Tomorrow is the last day to apply for the PSL...,original,NaN,NaN,...,7383,4093,NaN,Tammy Duckworth,10962,NaN,http://duckworth.senate.gov,True,policecarlight Tomorrow is the last day to ap...,575
3,585,1586730011075264515,https://twitter.com/SenatorHassan/status/15867...,Sun Oct 30 14:41:29 +0000 2022,2022-10-30T14:41:29+00:00,SenatorHassan,For-profit colleges have targeted veterans whi...,original,NaN,NaN,...,550,1746,NaN,Sen. Maggie Hassan,12133,NaN,http://hassan.senate.gov,True,Forprofit colleges have targeted veterans whil...,497
4,1080,1574771628977913857,https://twitter.com/salviforsenate/status/1574...,Tue Sep 27 14:43:08 +0000 2022,2022-09-27T14:43:08+00:00,salviforsenate,The CBO projects @TammyDuckworth’s support of ...,original,NaN,stopthespending reduceinflation protectfamilie...,...,221,21,NaN,Kathy Salvi,577,NaN,http://kathysalvi.com,True,The CBO projects TammyDuckworths support of st...,575


In [10]:
text = climate_tweets['clean_text'][0]
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

In [11]:
ranking = np.argsort(scores)
for i in range(scores.shape[0]):
    s = scores[ranking[i]]
    print(i, s)

0 0.016072957
1 0.039206844
2 0.94472027


In [7]:
sentiment_analysis_task = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [33]:
sentiment_list = [sentiment_analysis_task(x)[0] for x in climate_tweets['clean_text']]

In [34]:
sentiment_df = pd.DataFrame().from_dict(sentiment_list).rename(columns={'label':'sentiment','score':'confidence'})
climate_sentiment_df = pd.concat([climate_tweets,sentiment_df], axis=1)
climate_sentiment_df.head()

,Unnamed: 0,id,tweet_url,created_at,parsed_created_at,user_screen_name,text,tweet_type,coordinates,hashtags,...,user_location,user_name,user_statuses_count,user_time_zone,user_urls,user_verified,clean_text,topic_id,sentiment,confidence
0,74.0,1.587227e+18,https://twitter.com/ChrisVanHollen/status/1587...,Mon Oct 31 23:34:32 +0000 2022,2022-10-31T23:34:32+00:00,ChrisVanHollen,Great to celebrate the launch of Montgomery Co...,original,NaN,NaN,...,"Maryland, USA",Senator Chris Van Hollen,7425.0,NaN,https://www.vanhollen.senate.gov/,True,Great to celebrate the launch of Montgomery Co...,615.0,Positive,0.944720
1,676.0,1.586459e+18,https://twitter.com/PattyMurray/status/1586459...,Sat Oct 29 20:46:28 +0000 2022,2022-10-29T20:46:28+00:00,PattyMurray,The climate crisis is one of the most serious ...,original,NaN,NaN,...,WA & DC,Senator Patty Murray,23406.0,NaN,http://murray.senate.gov/,True,The climate crisis is one of the most serious ...,26.0,Positive,0.767403
2,772.0,1.586416e+18,https://twitter.com/RepValDemings/status/15864...,Sat Oct 29 17:53:52 +0000 2022,2022-10-29T17:53:52+00:00,RepValDemings,Florida families cannot continue to have their...,original,NaN,NaN,...,"Washington, DC I Orlando, FL",Rep. Val Demings,14028.0,NaN,http://demings.house.gov,True,Florida families cannot continue to have their...,26.0,Positive,0.705050
4,1560.0,1.572734e+18,https://twitter.com/TammyDuckworth/status/1572...,Wed Sep 21 23:46:03 +0000 2022,2022-09-21T23:46:03+00:00,TammyDuckworth,🚨🌎This is a BIG deal for our planet. The Kiga...,quote,NaN,NaN,...,Illinois,Tammy Duckworth,4651.0,NaN,http://tammyduckworth.com/,True,policecarlight globeshowingAmericas This is ...,26.0,Positive,0.887755
5,1606.0,1.562183e+18,https://twitter.com/TammyDuckworth/status/1562...,Tue Aug 23 21:01:10 +0000 2022,2022-08-23T21:01:10+00:00,TammyDuckworth,Students like @isudems are the future of our s...,original,NaN,NaN,...,Illinois,Tammy Duckworth,4651.0,NaN,http://tammyduckworth.com/,True,Students like isudems are the future of our st...,26.0,Negative,0.905484


In [35]:
sentiment_list = [sentiment_analysis_task(x)[0] for x in loan_tweets['clean_text']]

In [36]:
sentiment_df = pd.DataFrame().from_dict(sentiment_list).rename(columns={'label':'sentiment','score':'confidence'})
loan_sentiment_df = pd.concat([loan_tweets,sentiment_df], axis=1)
loan_sentiment_df.head()

,Unnamed: 0,id,tweet_url,created_at,parsed_created_at,user_screen_name,text,tweet_type,coordinates,hashtags,...,user_location,user_name,user_statuses_count,user_time_zone,user_urls,user_verified,clean_text,topic_id,sentiment,confidence
0,23.0,1.587124e+18,https://twitter.com/PattyMurray/status/1587124...,Mon Oct 31 16:47:39 +0000 2022,2022-10-31T16:47:39+00:00,PattyMurray,TODAY is the last day to apply for Public Serv...,quote,NaN,NaN,...,WA & DC,Senator Patty Murray,23410.0,NaN,http://murray.senate.gov/,True,TODAY is the last day to apply for Public Serv...,575.0,Neutral,0.920865
1,427.0,1.587157e+18,https://twitter.com/JamesForAR/status/15871571...,Mon Oct 31 18:58:48 +0000 2022,2022-10-31T18:58:48+00:00,JamesForAR,Today is the LAST DAY to apply for the Public ...,retweet,NaN,NaN,...,"Arkansas, USA",Natalie James for U.S. Senate,3424.0,NaN,http://JamesforArkansas.com,True,Today is the LAST DAY to apply for the Public ...,575.0,Neutral,0.893615
2,471.0,1.586761e+18,https://twitter.com/SenDuckworth/status/158676...,Sun Oct 30 16:46:19 +0000 2022,2022-10-30T16:46:19+00:00,SenDuckworth,🚨Tomorrow is the last day to apply for the PSL...,original,NaN,NaN,...,NaN,Tammy Duckworth,10962.0,NaN,http://duckworth.senate.gov,True,policecarlight Tomorrow is the last day to ap...,575.0,Neutral,0.775948
3,585.0,1.586730e+18,https://twitter.com/SenatorHassan/status/15867...,Sun Oct 30 14:41:29 +0000 2022,2022-10-30T14:41:29+00:00,SenatorHassan,For-profit colleges have targeted veterans whi...,original,NaN,NaN,...,NaN,Sen. Maggie Hassan,12133.0,NaN,http://hassan.senate.gov,True,Forprofit colleges have targeted veterans whil...,497.0,Negative,0.434776
4,1080.0,1.574772e+18,https://twitter.com/salviforsenate/status/1574...,Tue Sep 27 14:43:08 +0000 2022,2022-09-27T14:43:08+00:00,salviforsenate,The CBO projects @TammyDuckworth’s support of ...,original,NaN,stopthespending reduceinflation protectfamilie...,...,NaN,Kathy Salvi,577.0,NaN,http://kathysalvi.com,True,The CBO projects TammyDuckworths support of st...,575.0,Neutral,0.335155


In [37]:
climate_sentiment_df.to_csv("./data/climate_sentiments_11_1.csv")
loan_sentiment_df.to_csv("./data/loan_sentiments_11_1.csv")